### Melon 100 chart 수집/분석/저장
* 100곡 노래의 제목,번호,노래상세정보url을 List에 저장하기 (정규표현식 사용)
* 노래의 상세정보를 추출해서 자료구조에 저장하고, json 파일로 저장하기
* json 파일을 Pandas의 read_json() 이용해서 DataFrame 객체로 저장하기
* DataFrame객체를 DB의 Table로 저장하기

In [1]:
import requests
from bs4 import BeautifulSoup
import re # Regular Expression
import pandas as pd
import json

In [ ]:
url = 'https://www.melon.com/chart/index.htm'
req_header = {
    'user-agent':''
}